In [ ]:
# The notebook presently uses test images and some testing tools from the test directory.
HOME = "../.." # path to the root of the project two levels up

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

In [ ]:
# Load packages
using IceFloeTracker: LopezAcosta2019, LopezAcosta2019Tiling, Watkins2025GitHub
using Images: segment_mean, labels_map, SegmentedImage, RGB

## Load the images

In [ ]:
dataset = Watkins2025GitHub(; ref="a451cd5e62a10309a9640fbbe6b32a236fcebc70")(c-> c.case_number == 6 && c.satellite == "terra")
case = first(dataset)
truecolor = RGB.(case.modis_truecolor) # TODO: remove RGB cast

In [ ]:
falsecolor = RGB.(case.modis_falsecolor) # TODO: remove RGB cast

In [ ]:
landmask = RGB.(case.modis_landmask) # TODO: remove RGB cast

## Run the segmentation algorithm

We instantiate the algorithm as an object with all of its parameters as follows:

In [ ]:
segmentation_algorithm = LopezAcosta2019()

If we wanted to modify the options, we could include those in the call above. 

In [ ]:
segments = segmentation_algorithm(truecolor, falsecolor, landmask)

To show the results with each segment marked using its mean color:

In [ ]:
map(i -> segment_mean(segments, i), labels_map(segments))

We can do the same with the falsecolor image:

In [ ]:
# Get the labels_map
segments_falsecolor = SegmentedImage(falsecolor, labels_map(segments))
map(i -> segment_mean(segments_falsecolor, i), labels_map(segments_falsecolor))

## Run the segmentation algorithm with tiling
We instantiate the algorithm as an object with all of its parameters as follows:

In [ ]:
segmentation_algorithm_with_tiling = LopezAcosta2019Tiling()

In [ ]:
segments = segmentation_algorithm_with_tiling(truecolor, falsecolor, landmask)

To show the results with each segment marked using its mean color:

In [ ]:
map(i -> segment_mean(segments, i), labels_map(segments))

We can do the same with the falsecolor image:

In [ ]:
# Get the labels_map
segments_falsecolor = SegmentedImage(falsecolor, labels_map(segments))
map(i -> segment_mean(segments_falsecolor, i), labels_map(segments_falsecolor))